# Pygformula pipeline with features identiying L

In [167]:
import numpy as np
import pygformula
from pygformula import ParametricGformula
from pygformula.interventions import static

In [168]:
import pandas as pd

In [169]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler

In [170]:
import seaborn as sns
import matplotlib.pyplot as plt

# Load MIMIC data

In [171]:
mimicdata = pd.read_parquet('../Dataset/mimic-iv/from_pipeline/ltm_grided_cleaned_5000_A_Y_D_SelfPipeline.parquet')

In [172]:
mimicdata.shape

(31837, 38)

In [173]:
mimicdata.columns

Index(['stay_id', 'grid_end', 'vent_mode__hours_since_last__last_12h',
       'temperature__mean__last_12h', 'heart_rate__mean__last_12h',
       'arterial_blood_pressure_mean__mean__last_12h',
       'fluid_out_urine__mean__last_12h', 'pco2_arterial__mean__last_12h',
       'respiratory_rate_measured__mean__last_12h',
       'o2_saturation__mean__last_12h', 'po2_arterial__mean__last_12h',
       'bicarbonate_arterial__last__last_12h',
       'activated_partial_thromboplastin_time__last__last_12h',
       'hemoglobin__last__last_12h', 'creatinine__last__last_12h',
       'ureum__last__last_12h', 'lactate__last__last_12h',
       'glasgow_coma_scale_total__last__last_12h', 'o2_flow__last__last_12h',
       'vent_mode__last__last_12h', 'raw_age', 'sex', 'raw_height',
       'raw_weight', 'unit_type', 'origin', 'los', 'intime', 'outtime',
       'death_time_from_intime', 'icu_mortality', 'death_abs_time',
       'mortality_after_discharge', 't', 'ref_time', 'A', 'D', 'Y'],
      dtype='ob

In [174]:
mimicdata.rename(columns={"stay_id": "admission_id", "t": "t0", "raw_age":"age", "raw_height":"height", "raw_weight":"weight"}, inplace=True)

In [175]:
mimicdata.vent_mode__last__last_12h = mimicdata.vent_mode__last__last_12h.astype("category")

In [176]:
mimicdata.vent_mode__last__last_12h.value_counts()

vent_mode__last__last_12h
unknown                27797
invasive_controlled     2966
invasive_assisted       1071
cancelled                  3
Name: count, dtype: int64

var = "vent_mode__hours_since_last__last_12h"
sns.kdeplot(data=mimicdata, x=var, fill=True, bw_adjust=1)
plt.title(var)
plt.tight_layout()
plt.show()

# Declare baseline variables and meta variables

In [177]:
#baseline = ['age',
#            'gender',
#            'ethnicity',
#            'insurance'
#            ]
baseline = ["age", 
            "sex", 
            #"height", 
            "weight", 
            #"unit_type", 
            "origin"]

In [178]:
# Pacmed - 7.8% have died inside the ICU
mimicdata.groupby('admission_id').tail(1)['D'].mean()

0.11921551344204495

In [179]:
# Pacmed - 3.3% have died outside the ICU within 7 days after discharge
mimicdata.groupby('admission_id').tail(1)['Y'].mean()

0.015011258443832875

In [180]:
# Pacmed - 92% have been discharged from the ICU. The rest died inside the ICU which tallied with 7.8% that have D=1
mimicdata.groupby('admission_id').tail(1)['A'].mean()

0.880784486557955

In [181]:
picked_L = [
    #'vent_status__last__last_12h',
    'vent_mode__last__last_12h',
    'vent_mode__hours_since_last__last_12h',
    
    'bicarbonate_arterial__last__last_12h',
    'activated_partial_thromboplastin_time__last__last_12h',
    'temperature__mean__last_12h',

    'hemoglobin__last__last_12h',
    'heart_rate__mean__last_12h',
    'arterial_blood_pressure_mean__mean__last_12h',

    'creatinine__last__last_12h',
    'ureum__last__last_12h',
    'fluid_out_urine__mean__last_12h',
    'lactate__last__last_12h',
    
    'glasgow_coma_scale_total__last__last_12h',
    
    'pco2_arterial__mean__last_12h',
    'respiratory_rate_measured__mean__last_12h',
    'o2_saturation__mean__last_12h',
    'o2_flow__last__last_12h',  
    'po2_arterial__mean__last_12h',
]

In [182]:
mimicdata.admission_id.nunique()

4538

In [183]:
mimicdata.groupby('admission_id').tail(1)['D'].mean()

0.11921551344204495

In [184]:
mimicdata.groupby('admission_id').tail(1)['Y'].mean()

0.015011258443832875

In [185]:
mimicdata.groupby('admission_id').tail(1)['Y'].isna().mean()

0.11921551344204495

In [186]:
mimicdata.groupby('admission_id').tail(1)['A'].mean()

0.880784486557955

## Handle missing values

In [187]:
# Columns to forward fill
feature_columns = list(picked_L)

In [188]:
mimicdata[feature_columns].isna().sum()

vent_mode__last__last_12h                                    0
vent_mode__hours_since_last__last_12h                    24187
bicarbonate_arterial__last__last_12h                     31031
activated_partial_thromboplastin_time__last__last_12h    31265
temperature__mean__last_12h                               2534
hemoglobin__last__last_12h                               31019
heart_rate__mean__last_12h                                  66
arterial_blood_pressure_mean__mean__last_12h              1001
creatinine__last__last_12h                               31031
ureum__last__last_12h                                    31034
fluid_out_urine__mean__last_12h                          10550
lactate__last__last_12h                                  31398
glasgow_coma_scale_total__last__last_12h                 27388
pco2_arterial__mean__last_12h                            19762
respiratory_rate_measured__mean__last_12h                  215
o2_saturation__mean__last_12h                          

In [189]:
mimicdata.loc[mimicdata.admission_id=="mimic4-39996123"][["grid_end", "vent_mode__last__last_12h", "vent_mode__hours_since_last__last_12h"]]

,grid_end,vent_mode__last__last_12h,vent_mode__hours_since_last__last_12h
31822,0 days 12:00:00,invasive_controlled,1.783333
31823,1 days 00:00:00,invasive_assisted,5.366667
31824,1 days 12:00:00,unknown,NaN
31825,2 days 00:00:00,unknown,NaN
31826,2 days 12:00:00,unknown,NaN
31827,3 days 00:00:00,unknown,NaN


In [190]:
# # Apply forward fill per group, only on selected columns; Then backward fill to handle top-level NaNs per group
mimicdata[feature_columns] = mimicdata.groupby('admission_id')[feature_columns].ffill().bfill()

In [191]:
mimicdata.loc[mimicdata.admission_id=="mimic4-39996123"][["grid_end", "vent_mode__last__last_12h", "vent_mode__hours_since_last__last_12h"]]

,grid_end,vent_mode__last__last_12h,vent_mode__hours_since_last__last_12h
31822,0 days 12:00:00,invasive_controlled,1.783333
31823,1 days 00:00:00,invasive_assisted,5.366667
31824,1 days 12:00:00,unknown,5.366667
31825,2 days 00:00:00,unknown,5.366667
31826,2 days 12:00:00,unknown,5.366667
31827,3 days 00:00:00,unknown,5.366667


In [192]:
mimicdata[feature_columns].isna().sum()

vent_mode__last__last_12h                                  0
vent_mode__hours_since_last__last_12h                      9
bicarbonate_arterial__last__last_12h                      42
activated_partial_thromboplastin_time__last__last_12h     42
temperature__mean__last_12h                                0
hemoglobin__last__last_12h                                15
heart_rate__mean__last_12h                                 0
arterial_blood_pressure_mean__mean__last_12h               0
creatinine__last__last_12h                                42
ureum__last__last_12h                                     42
fluid_out_urine__mean__last_12h                            0
lactate__last__last_12h                                  291
glasgow_coma_scale_total__last__last_12h                   0
pco2_arterial__mean__last_12h                              9
respiratory_rate_measured__mean__last_12h                  0
o2_saturation__mean__last_12h                              0
o2_flow__last__last_12h 

In [193]:
# Find rows where any of these columns are NaN
mask_missing = mimicdata[feature_columns].isna().any(axis=1)

# Get corresponding admission IDs
missing_admissions = mimicdata.loc[mask_missing, "admission_id"].unique()

print("Admissions with any missing feature:", missing_admissions)
print("Total:", len(missing_admissions))


Admissions with any missing feature: ['mimic4-39934716' 'mimic4-39935591' 'mimic4-39936799' 'mimic4-39940089'
 'mimic4-39941251' 'mimic4-39942872' 'mimic4-39947104' 'mimic4-39949224'
 'mimic4-39950441' 'mimic4-39950631' 'mimic4-39951743' 'mimic4-39954437'
 'mimic4-39955694' 'mimic4-39956085' 'mimic4-39960907' 'mimic4-39961682'
 'mimic4-39962257' 'mimic4-39963902' 'mimic4-39965448' 'mimic4-39966506'
 'mimic4-39966638' 'mimic4-39969718' 'mimic4-39982305' 'mimic4-39984173'
 'mimic4-39984228' 'mimic4-39984456' 'mimic4-39987417' 'mimic4-39990055'
 'mimic4-39990748' 'mimic4-39993298' 'mimic4-39993560' 'mimic4-39996044'
 'mimic4-39996123' 'mimic4-39997955']
Total: 34


In [194]:
mimicdata.loc[mimicdata['admission_id']=='mimic4-39941251'].isna().sum()

admission_id                                             0
grid_end                                                 0
vent_mode__hours_since_last__last_12h                    0
temperature__mean__last_12h                              0
heart_rate__mean__last_12h                               0
arterial_blood_pressure_mean__mean__last_12h             0
fluid_out_urine__mean__last_12h                          0
pco2_arterial__mean__last_12h                            0
respiratory_rate_measured__mean__last_12h                0
o2_saturation__mean__last_12h                            0
po2_arterial__mean__last_12h                             0
bicarbonate_arterial__last__last_12h                     0
activated_partial_thromboplastin_time__last__last_12h    0
hemoglobin__last__last_12h                               0
creatinine__last__last_12h                               0
ureum__last__last_12h                                    0
lactate__last__last_12h                                 

In [195]:
mimicdata = mimicdata[~mimicdata['admission_id'].isin(missing_admissions)]

# Check missingness including baseline variables

In [196]:
baseline = ["age", 
            "sex", 
            #"height", 
            #"weight", 
            #"unit_type", 
            "origin"]

def fill_baseline_first_valid(df, baseline_vars):
    df_out = df.copy()

    # For each baseline variable
    for col in baseline_vars:
        # Get first non-NaN value per admission_id
        first_valid = (
            df_out.groupby("admission_id")[col]
                  .apply(lambda s: s.dropna().iloc[0] if s.dropna().size > 0 else np.nan)
        )

        # Map back to all rows
        df_out[col] = df_out["admission_id"].map(first_valid)

    return df_out
mimicdata = fill_baseline_first_valid(mimicdata, baseline)

In [197]:
#check missingness including baseline variables
all = feature_columns + baseline
mimicdata[all].isna().sum()

vent_mode__last__last_12h                                   0
vent_mode__hours_since_last__last_12h                       0
bicarbonate_arterial__last__last_12h                        0
activated_partial_thromboplastin_time__last__last_12h       0
temperature__mean__last_12h                                 0
hemoglobin__last__last_12h                                  0
heart_rate__mean__last_12h                                  0
arterial_blood_pressure_mean__mean__last_12h                0
creatinine__last__last_12h                                  0
ureum__last__last_12h                                       0
fluid_out_urine__mean__last_12h                             0
lactate__last__last_12h                                     0
glasgow_coma_scale_total__last__last_12h                    0
pco2_arterial__mean__last_12h                               0
respiratory_rate_measured__mean__last_12h                   0
o2_saturation__mean__last_12h                               0
o2_flow_

In [198]:
# Find rows where any of these columns are NaN
mask_missing = mimicdata[all].isna().any(axis=1)

# Get corresponding admission IDs
missing_admissions = mimicdata.loc[mask_missing, "admission_id"].unique()

print("Admissions with any missing feature:", missing_admissions)
print("Total:", len(missing_admissions))

Admissions with any missing feature: ['mimic4-30089616' 'mimic4-30121250' 'mimic4-30215337' 'mimic4-30247538'
 'mimic4-30308836' 'mimic4-30350342' 'mimic4-30358483' 'mimic4-30475901'
 'mimic4-30485433' 'mimic4-30496342' 'mimic4-30499042' 'mimic4-30594401'
 'mimic4-30635125' 'mimic4-30781525' 'mimic4-30786514' 'mimic4-30805749'
 'mimic4-30855082' 'mimic4-30901960' 'mimic4-30927811' 'mimic4-30936874'
 'mimic4-30944047' 'mimic4-31058150' 'mimic4-31222007' 'mimic4-31235235'
 'mimic4-31264381' 'mimic4-31337510' 'mimic4-31370025' 'mimic4-31468973'
 'mimic4-31484900' 'mimic4-31539713' 'mimic4-31679411' 'mimic4-31728367'
 'mimic4-31740877' 'mimic4-31784179' 'mimic4-31817362' 'mimic4-31819927'
 'mimic4-31820960' 'mimic4-31868604' 'mimic4-31933584' 'mimic4-32008832'
 'mimic4-32119264' 'mimic4-32152938' 'mimic4-32161483' 'mimic4-32192065'
 'mimic4-32358682' 'mimic4-32410153' 'mimic4-32423195' 'mimic4-32490492'
 'mimic4-32565831' 'mimic4-32567201' 'mimic4-32651167' 'mimic4-32703143'
 'mimic4-32708

In [199]:
mimicdata.loc[mimicdata.admission_id=="mimic4-35155921"].sex

17006   NaN
17007   NaN
17008   NaN
17009   NaN
17010   NaN
17011   NaN
17012   NaN
17013   NaN
17014   NaN
17015   NaN
17016   NaN
17017   NaN
Name: sex, dtype: float64

In [200]:
mimicdata = mimicdata[~mimicdata['admission_id'].isin(missing_admissions)]

In [201]:
mimicdata.loc[mimicdata.admission_id=="mimic4-35155921"]

,admission_id,grid_end,vent_mode__hours_since_last__last_12h,temperature__mean__last_12h,heart_rate__mean__last_12h,arterial_blood_pressure_mean__mean__last_12h,fluid_out_urine__mean__last_12h,pco2_arterial__mean__last_12h,respiratory_rate_measured__mean__last_12h,o2_saturation__mean__last_12h,...,outtime,death_time_from_intime,icu_mortality,death_abs_time,mortality_after_discharge,t0,ref_time,A,D,Y


In [202]:
mimicdata.admission_id.nunique()

4327

In [203]:
mimicdata[all].isna().sum()

vent_mode__last__last_12h                                0
vent_mode__hours_since_last__last_12h                    0
bicarbonate_arterial__last__last_12h                     0
activated_partial_thromboplastin_time__last__last_12h    0
temperature__mean__last_12h                              0
hemoglobin__last__last_12h                               0
heart_rate__mean__last_12h                               0
arterial_blood_pressure_mean__mean__last_12h             0
creatinine__last__last_12h                               0
ureum__last__last_12h                                    0
fluid_out_urine__mean__last_12h                          0
lactate__last__last_12h                                  0
glasgow_coma_scale_total__last__last_12h                 0
pco2_arterial__mean__last_12h                            0
respiratory_rate_measured__mean__last_12h                0
o2_saturation__mean__last_12h                            0
o2_flow__last__last_12h                                 

In [204]:
mimicdata['t0'] = mimicdata['t0'].astype(int)

In [205]:
mimicdata.Y

0        NaN
1        0.0
2        NaN
3        NaN
4        NaN
        ... 
31541    NaN
31542    NaN
31543    0.0
31544    NaN
31545    0.0
Name: Y, Length: 30214, dtype: float64

# Normalizing numeric columns

In [206]:
to_norm = [
 'vent_mode__hours_since_last__last_12h',
 'pco2_arterial__mean__last_12h',
 'po2_arterial__mean__last_12h',
 'o2_flow__last__last_12h',
 'o2_saturation__mean__last_12h',
 'respiratory_rate_measured__mean__last_12h',

 'glasgow_coma_scale_total__last__last_12h',
 'lactate__last__last_12h',

 'fluid_out_urine__mean__last_12h',
 'ureum__last__last_12h',
 'creatinine__last__last_12h',

 'arterial_blood_pressure_mean__mean__last_12h',
 'heart_rate__mean__last_12h',
 'hemoglobin__last__last_12h',
    
 'temperature__mean__last_12h',
 'activated_partial_thromboplastin_time__last__last_12h', # time for blood to clot
 'bicarbonate_arterial__last__last_12h'
]

In [207]:
scaler = StandardScaler()
mimicdata_scaled = scaler.fit_transform(mimicdata[to_norm])
mimicdata_scaled_df = pd.DataFrame(mimicdata_scaled, columns=to_norm, index=mimicdata.index)
mimicdata[mimicdata_scaled_df.columns] = mimicdata_scaled_df

In [208]:
los_per_admission = mimicdata.groupby("admission_id")["t0"].max().reset_index(name="LOS")
los_per_admission["LOS"] = los_per_admission["LOS"] + 1

In [209]:
los_per_admission

,admission_id,LOS
0,mimic4-30002548,2
1,mimic4-30003372,4
2,mimic4-30006565,6
3,mimic4-30007983,3
4,mimic4-30009597,12
...,...,...
4322,mimic4-39919197,34
4323,mimic4-39922538,1
4324,mimic4-39930981,12
4325,mimic4-39931174,7


In [210]:
unique_los = los_per_admission["LOS"].unique()
unique_los.sort()
print(unique_los)


[  1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17  18
  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35  36
  37  38  39  40  41  42  43  44  45  47  48  49  50  51  52  53  55  56
  57  58  59  60  63  65  66  67  68  69  74  76  79  85  86  87  96  97
 100 101 109 111 117 181]


In [211]:
# LOS per admission (assuming t0 starts at 0)
los_per_admission = (
    mimicdata.groupby("admission_id")["t0"].max().reset_index(name="LOS")
)
los_per_admission["LOS"] = los_per_admission["LOS"] + 1

# Distribution of LOS
los_distribution = los_per_admission["LOS"].value_counts().sort_index()

print(los_distribution)


LOS
1      995
2      669
3      572
4      319
5      332
      ... 
101      1
109      1
111      1
117      1
181      1
Name: count, Length: 78, dtype: int64


In [212]:
los_distribution.sort_values()

LOS
181      1
47       1
55       1
56       1
57       1
      ... 
4      319
5      332
3      572
2      669
1      995
Name: count, Length: 78, dtype: int64

In [213]:
los_distribution.sort_values().to_csv("los_dist.csv", index=True)

plt.figure(figsize=(8,5))
plt.hist(los_per_admission["LOS"], bins=50, edgecolor='black')
plt.xlabel("LOS")
plt.ylabel("Count")
plt.title("Distribution of Length of Stay (LOS)")
plt.tight_layout()
plt.show()

In [214]:
#mimicdata = mimicdata[mimicdata["los"] <= pd.to_timedelta(15, unit="D")].copy()

In [215]:
mimicdata.admission_id.nunique()

4327

# Applying g-formula

In [216]:
time_name = 't0'
id_ = 'admission_id'

In [217]:
basecovs = baseline
'''['age',
            'gender',
            'ethnicity',
            'insurance'
            ]'''

"['age',\n            'gender',\n            'ethnicity',\n            'insurance'\n            ]"

In [218]:
covnames = [
    #'vent_status__last__last_12h',
    'vent_mode__last__last_12h',
    'vent_mode__hours_since_last__last_12h',
    
    'pco2_arterial__mean__last_12h',
    'po2_arterial__mean__last_12h',
    'o2_flow__last__last_12h',
    'o2_saturation__mean__last_12h',
    'respiratory_rate_measured__mean__last_12h',
    
    'glasgow_coma_scale_total__last__last_12h',

    'lactate__last__last_12h',
    'fluid_out_urine__mean__last_12h',
    'ureum__last__last_12h',
    'creatinine__last__last_12h',

    'arterial_blood_pressure_mean__mean__last_12h',
    'heart_rate__mean__last_12h',
    'hemoglobin__last__last_12h',
    
    'temperature__mean__last_12h',
    'activated_partial_thromboplastin_time__last__last_12h',
    'bicarbonate_arterial__last__last_12h',
    
    'A'
]



In [219]:
covtypes = [
 'categorical',          #'vent_mode__last__last_12h'
 'normal',#'zero-inflated normal', #'vent_mode__hours_since_last__last_12h'

 'normal',               #'pco2_arterial__mean__last_12h'
 'normal',               #'po2_arterial__mean__last_12h'
 'normal',               #'o2_flow__last__last_12h',
 'normal',               #'o2_saturation__mean__last_12h',
 'normal',               #'respiratory_rate_measured__mean__last_12h',

 'normal', #'categorical',          #'glasgow_coma_scale_total__last__last_12h',
 'normal',               #'lactate__last__last_12h',

 'normal',               #'fluid_out_urine__mean__last_12h',
 'normal',               #'ureum__last__last_12h',
 'normal',               #'creatinine__last__last_12h',

 'normal',               #'arterial_blood_pressure_mean__mean__last_12h',
 'normal',               #'heart_rate__mean__last_12h',
 'normal',               #'hemoglobin__last__last_12h'
    
 'normal',               #'temperature__mean__last_12h',
 'normal',               #'activated_partial_thromboplastin_time__last__last_12h',
 'normal',               #'bicarbonate_arterial__last__last_12h'

 'binary'              #'A'
]

In [220]:
covmodels = [
 #'vent_status__last__last_12h ~ lag1_vent_status__last__last_12h + glasgow_coma_scale_total__last__last_12h + po2_arterial__mean__last_12h + o2_saturation__mean__last_12h + respiratory_rate_measured__mean__last_12h',
  
 'vent_mode__last__last_12h ~ glasgow_coma_scale_total__last__last_12h + po2_arterial__mean__last_12h + o2_saturation__mean__last_12h + respiratory_rate_measured__mean__last_12h + t0',
 #'vent_status__last__last_12h ~ po2_arterial__mean__last_12h + o2_saturation__mean__last_12h + respiratory_rate_measured__mean__last_12h',
    
 'vent_mode__hours_since_last__last_12h ~ lag1_vent_mode__hours_since_last__last_12h + vent_mode__last__last_12h + t0',

 'pco2_arterial__mean__last_12h ~ lag1_pco2_arterial__mean__last_12h + o2_flow__last__last_12h + t0', 
 'po2_arterial__mean__last_12h ~ lag1_po2_arterial__mean__last_12h + o2_flow__last__last_12h + pco2_arterial__mean__last_12h + t0',
 'o2_flow__last__last_12h ~ lag1_o2_flow__last__last_12h + vent_mode__last__last_12h + respiratory_rate_measured__mean__last_12h + t0',
 'o2_saturation__mean__last_12h ~ lag1_o2_saturation__mean__last_12h + po2_arterial__mean__last_12h + respiratory_rate_measured__mean__last_12h + t0',
 
 'respiratory_rate_measured__mean__last_12h ~ lag1_respiratory_rate_measured__mean__last_12h + glasgow_coma_scale_total__last__last_12h + t0',
 #'respiratory_rate_measured__mean__last_12h ~ lag1_respiratory_rate_measured__mean__last_12h',

 'glasgow_coma_scale_total__last__last_12h ~ lag1_glasgow_coma_scale_total__last__last_12h + arterial_blood_pressure_mean__mean__last_12h + lactate__last__last_12h + t0',
 
 'lactate__last__last_12h ~ lag1_lactate__last__last_12h + arterial_blood_pressure_mean__mean__last_12h + fluid_out_urine__mean__last_12h + t0',
 #'lactate__last__last_12h ~ lag1_lactate__last__last_12h + arterial_blood_pressure_mean__mean__last_12h',

 'fluid_out_urine__mean__last_12h ~ lag1_fluid_out_urine__mean__last_12h + arterial_blood_pressure_mean__mean__last_12h + ureum__last__last_12h + t0',
 'ureum__last__last_12h ~ lag1_ureum__last__last_12h + creatinine__last__last_12h + t0',
 'creatinine__last__last_12h ~ lag1_creatinine__last__last_12h + arterial_blood_pressure_mean__mean__last_12h + t0',

 'arterial_blood_pressure_mean__mean__last_12h ~ lag1_arterial_blood_pressure_mean__mean__last_12h + heart_rate__mean__last_12h + bicarbonate_arterial__last__last_12h + t0',
 'heart_rate__mean__last_12h ~ lag1_heart_rate__mean__last_12h + temperature__mean__last_12h + hemoglobin__last__last_12h + t0',
 'hemoglobin__last__last_12h ~ lag1_hemoglobin__last__last_12h + activated_partial_thromboplastin_time__last__last_12h + t0',
    
 'temperature__mean__last_12h ~ lag1_temperature__mean__last_12h + t0',
 'activated_partial_thromboplastin_time__last__last_12h ~ lag1_activated_partial_thromboplastin_time__last__last_12h + t0',
 'bicarbonate_arterial__last__last_12h ~ lag1_bicarbonate_arterial__last__last_12h + t0',

 
 'A ~ lag1_A + vent_mode__last__last_12h + vent_mode__hours_since_last__last_12h + po2_arterial__mean__last_12h + o2_flow__last__last_12h + o2_saturation__mean__last_12h + respiratory_rate_measured__mean__last_12h + glasgow_coma_scale_total__last__last_12h + lactate__last__last_12h + fluid_out_urine__mean__last_12h + ureum__last__last_12h + creatinine__last__last_12h + arterial_blood_pressure_mean__mean__last_12h + heart_rate__mean__last_12h + hemoglobin__last__last_12h + temperature__mean__last_12h + activated_partial_thromboplastin_time__last__last_12h + bicarbonate_arterial__last__last_12h + t0']
 
 #'A ~ lag1_A + vent_mode__last__last_12h + vent_status__hours_since_last__last_12h + po2_arterial__mean__last_12h + o2_flow__last__last_12h + o2_saturation__mean__last_12h + respiratory_rate_measured__mean__last_12h + glasgow_coma_scale_total__last__last_12h + lactate__last__last_12h + fluid_out_urine__priority__mean__last_12h + ureum__last__last_12h + creatinine__last__last_12h + arterial_blood_pressure_mean__mean__last_12h + heart_rate__mean__last_12h + hemoglobin__last__last_12h + temperature__mean__last_12h + activated_partial_thromboplastin_time__last__last_12h + bicarbonate_arterial__last__last_12h + t0']

 #'A ~ lag1_A + vent_status__last__last_12h + po2_arterial__mean__last_12h + o2_flow__last__last_12h + o2_saturation__mean__last_12h + respiratory_rate_measured__mean__last_12h + glasgow_coma_scale_total__last__last_12h + lactate__last__last_12h + fluid_out_urine__priority__mean__last_12h + ureum__last__last_12h + creatinine__last__last_12h + arterial_blood_pressure_mean__mean__last_12h + heart_rate__mean__last_12h + hemoglobin__last__last_12h + temperature__mean__last_12h + activated_partial_thromboplastin_time__last__last_12h + bicarbonate_arterial__last__last_12h + t0']

 #'A ~ lag1_A + vent_status__last__last_12h + vent_status__hours_since_last__last_12h + po2_arterial__mean__last_12h + o2_flow__last__last_12h + o2_saturation__mean__last_12h + respiratory_rate_measured__mean__last_12h + lactate__last__last_12h + ureum__last__last_12h + creatinine__last__last_12h + arterial_blood_pressure_mean__mean__last_12h + heart_rate__mean__last_12h + hemoglobin__last__last_12h + temperature__mean__last_12h + activated_partial_thromboplastin_time__last__last_12h + bicarbonate_arterial__last__last_12h + t0']


In [221]:
outcome_name = 'Y'
# Outcome model
ymodel = 'Y ~ A + vent_mode__last__last_12h + cumavg_vent_mode__hours_since_last__last_12h + cumavg_pco2_arterial__mean__last_12h + cumavg_po2_arterial__mean__last_12h + cumavg_o2_flow__last__last_12h + cumavg_o2_saturation__mean__last_12h + cumavg_respiratory_rate_measured__mean__last_12h + cumavg_glasgow_coma_scale_total__last__last_12h + cumavg_lactate__last__last_12h + cumavg_fluid_out_urine__mean__last_12h + cumavg_ureum__last__last_12h + cumavg_creatinine__last__last_12h + cumavg_arterial_blood_pressure_mean__mean__last_12h + cumavg_heart_rate__mean__last_12h + cumavg_hemoglobin__last__last_12h + cumavg_temperature__mean__last_12h + cumavg_activated_partial_thromboplastin_time__last__last_12h + cumavg_bicarbonate_arterial__last__last_12h + t0'
#ymodel = 'Y ~ A + vent_mode__last__last_12h + cumavg_vent_status__hours_since_last__last_12h + cumavg_pco2_arterial__mean__last_12h + cumavg_po2_arterial__mean__last_12h + cumavg_o2_flow__last__last_12h + cumavg_o2_saturation__mean__last_12h + cumavg_respiratory_rate_measured__mean__last_12h + cumavg_glasgow_coma_scale_total__last__last_12h + cumavg_lactate__last__last_12h + cumavg_fluid_out_urine__priority__mean__last_12h + cumavg_ureum__last__last_12h + cumavg_creatinine__last__last_12h + cumavg_arterial_blood_pressure_mean__mean__last_12h + cumavg_heart_rate__mean__last_12h + cumavg_hemoglobin__last__last_12h + cumavg_temperature__mean__last_12h + cumavg_activated_partial_thromboplastin_time__last__last_12h + cumavg_bicarbonate_arterial__last__last_12h + t0'
#ymodel = 'Y ~ A + vent_status__last__last_12h + cumavg_pco2_arterial__mean__last_12h + cumavg_po2_arterial__mean__last_12h + cumavg_o2_flow__last__last_12h + cumavg_o2_saturation__mean__last_12h + cumavg_respiratory_rate_measured__mean__last_12h + cumavg_lactate__last__last_12h + cumavg_ureum__last__last_12h + cumavg_creatinine__last__last_12h + cumavg_arterial_blood_pressure_mean__mean__last_12h + cumavg_heart_rate__mean__last_12h + cumavg_hemoglobin__last__last_12h + cumavg_temperature__mean__last_12h + cumavg_activated_partial_thromboplastin_time__last__last_12h + cumavg_bicarbonate_arterial__last__last_12h + t0'

outcome_type = 'binary_eof'

In [222]:
censor_name = 'D'

censor_model = 'D ~ A + vent_mode__last__last_12h + cumavg_vent_mode__hours_since_last__last_12h + pco2_arterial__mean__last_12h + po2_arterial__mean__last_12h + o2_flow__last__last_12h + o2_saturation__mean__last_12h + respiratory_rate_measured__mean__last_12h + glasgow_coma_scale_total__last__last_12h + lactate__last__last_12h + fluid_out_urine__mean__last_12h + ureum__last__last_12h + creatinine__last__last_12h + arterial_blood_pressure_mean__mean__last_12h + heart_rate__mean__last_12h + hemoglobin__last__last_12h + temperature__mean__last_12h + activated_partial_thromboplastin_time__last__last_12h + bicarbonate_arterial__last__last_12h + t0'
#censor_model = 'D ~ A + vent_status__last__last_12h + vent_status__hours_since_last__last_12h + pco2_arterial__mean__last_12h + po2_arterial__mean__last_12h + o2_flow__last__last_12h + o2_saturation__mean__last_12h + respiratory_rate_measured__mean__last_12h + glasgow_coma_scale_total__last__last_12h + lactate__last__last_12h + fluid_out_urine__priority__mean__last_12h + ureum__last__last_12h + creatinine__last__last_12h + arterial_blood_pressure_mean__mean__last_12h + heart_rate__mean__last_12h + hemoglobin__last__last_12h + temperature__mean__last_12h + activated_partial_thromboplastin_time__last__last_12h + bicarbonate_arterial__last__last_12h + t0'
#censor_model = 'D ~ A + vent_status__last__last_12h + vent_status__hours_since_last__last_12h + pco2_arterial__mean__last_12h + po2_arterial__mean__last_12h + o2_flow__last__last_12h + o2_saturation__mean__last_12h + respiratory_rate_measured__mean__last_12h  + lactate__last__last_12h  + ureum__last__last_12h + creatinine__last__last_12h + arterial_blood_pressure_mean__mean__last_12h + heart_rate__mean__last_12h + hemoglobin__last__last_12h + temperature__mean__last_12h + activated_partial_thromboplastin_time__last__last_12h + bicarbonate_arterial__last__last_12h + t0'
#compevent_cens = False

In [223]:
#time_points = np.max(np.unique(mimicdata[time_name])) + 1
time_points = 15

In [224]:
#int_descript = ['Discharge 0th point', 'Discharge 1st point', 'Discharge 2nd point', 'Discharge 3rd point', 'Discharge 4th point', 'Discharge 5th point', 'Discharge 6th point']

#int_descript = ['Discharge 6th point', 'Discharge natural course', 'Discharge safe', 'Discharge risky', 'Discharge dangerous']
#int_descript = ['Discharge extremely dangerous']
int_descript = ['Discharge 3rd day']

'''Intervention1_A = [static, np.array([1]), [0]]
Intervention2_A = [static, np.array([0, 1]), [0, 1]]
Intervention3_A = [static, np.array([0, 0, 1]), [0, 1, 2]]
Intervention4_A = [static, np.array([0, 0, 0, 1]), [0, 1, 2, 3]]
Intervention5_A = [static, np.array([0, 0, 0, 0, 1]), [0, 1, 2, 3, 4]]
Intervention6_A = [static, np.array([0, 0, 0, 0, 0, 1]), [0, 1, 2, 3, 4, 5]]
Intervention7_A = [static, np.array([0, 0, 0, 0, 0, 0, 1]), [0, 1, 2, 3, 4, 5, 6]]'''

#Intervention1_A = [static, np.array([0, 0, 0, 0, 0, 0,0,0,0,0,1]), [0, 1, 2, 3, 4, 5, 6,7, 8, 9,10]]
Intervention1_A = [static, np.array([0, 0, 0, 0, 0, 1]), [0, 1, 2, 3, 4, 5]]

'''#int_descript = ['Dynamic intervention']
def dynamic_intervention(new_df, pool, int_var, time_name, t):
    new_df.loc[new_df[time_name] == t, int_var] = 0
    new_df.loc[new_df['glasgow_coma_scale_total__last__last_12h'] < 5, int_var] = 1

# Natural course for test
def dynamic_NC(new_df, pool, int_var, time_name, t):
    new_df.loc[new_df[time_name] == t, int_var] = 0
    # Set int_var = 1 if all variables are within their respective observed ranges
    condition = (
        (new_df['vent_mode__last__last_12h'].isin(['missing', 'invasive_assisted', 'invasive_controlled', 'niv_unknown', 'niv_assisted'])) &
        (new_df['vent_mode__hours_since_last__last_12h'].between(0.0, 12.0)) &
        (new_df['pco2_arterial__mean__last_12h'].between(15.49, 86.0)) &
        (new_df['po2_arterial__mean__last_12h'].between(16.0, 463.83)) &
        (new_df['o2_flow__last__last_12h'].between(0.0, 38.63)) &
        (new_df['o2_saturation__mean__last_12h'].between(86.10, 100.0)) &
        (new_df['respiratory_rate_measured__mean__last_12h'].between(8.23, 32.94)) &
        (new_df['glasgow_coma_scale_total__last__last_12h'].between(3.0, 15.0)) &
        (new_df['lactate__last__last_12h'].between(0.2, 20.17)) &
        (new_df['fluid_out_urine__total_value_extrapolated__last_12h'].between(0.0, 28631.58)) &
        (new_df['ureum__last__last_12h'].between(1.0, 156.84)) &
        (new_df['creatinine__last__last_12h'].between(0.0, 10.0)) &
        (new_df['arterial_blood_pressure_mean__mean__last_12h'].between(34.32, 166.71)) &
        (new_df['heart_rate__mean__last_12h'].between(40.01, 137.99)) &
        (new_df['hemoglobin__last__last_12h'].between(5.38, 18.16)) &
        (new_df['temperature__mean__last_12h'].between(34.48, 38.83)) &
        (new_df['activated_partial_thromboplastin_time__last__last_12h'].between(15.9, 128.46)) &
        (new_df['bicarbonate_arterial__last__last_12h'].between(10.04, 46.77))
    )
    new_df.loc[condition, int_var] = 1

# Safe
def dynamic_safe(new_df, pool, int_var, time_name, t):
    new_df.loc[new_df[time_name] == t, int_var] = 0
    # Safe discharge: patients are very stable and ready for ICU discharge
    condition_safe = (
        # Ventilation: ideally off support – “missing” means no active ventilation record.
        (new_df['vent_mode__last__last_12h'].isin(['missing'])) &  # safe if not on mechanical ventilation
        # Ventilation time: more than 8 hours since last support to ensure stability.
        (new_df['vent_mode__hours_since_last__last_12h'].between(8.0, 12.0)) &  # safe if off ventilation for >=8h
        # pCO2: normal range indicating effective ventilation.
        (new_df['pco2_arterial__mean__last_12h'].between(35.0, 45.0)) &  # normocapnia is desirable
        # pO2: adequate oxygenation.
        (new_df['po2_arterial__mean__last_12h'].between(80.0, 300.0)) &  # well-oxygenated patients
        # Oxygen flow: low supplemental oxygen requirements.
        (new_df['o2_flow__last__last_12h'].between(0.0, 8.0)) &  # minimal oxygen support
        # O2 saturation: near-optimal saturation.
        (new_df['o2_saturation__mean__last_12h'].between(95.0, 100.0)) &  # saturation in ideal range
        # Respiratory rate: within a normal breathing range.
        (new_df['respiratory_rate_measured__mean__last_12h'].between(12.0, 20.0)) &  # normal respiratory rate
        # Neurological status: fully alert (or nearly so).
        (new_df['glasgow_coma_scale_total__last__last_12h'].between(14.0, 15.0)) &  # high GCS reflects alertness
        # Lactate: low levels indicate good tissue perfusion.
        (new_df['lactate__last__last_12h'] < 2.0) &  # low lactate is favorable
        # Urine output: robust output suggests adequate kidney function.
        (new_df['fluid_out_urine__total_value_extrapolated__last_12h'] >= 3000) &  # higher urine output is reassuring
        # Ureum: low blood urea levels.
        (new_df['ureum__last__last_12h'] < 40.0) &  # lower ureum points to good renal clearance
        # Creatinine: low levels indicate preserved kidney function.
        (new_df['creatinine__last__last_12h'] < 1.5) &  # creatinine in safe range
        # Mean arterial blood pressure: stable hemodynamics.
        (new_df['arterial_blood_pressure_mean__mean__last_12h'].between(70.0, 100.0)) &  # adequate blood pressure
        # Heart rate: within normal limits.
        (new_df['heart_rate__mean__last_12h'].between(60.0, 100.0)) &  # heart rate in safe zone
        # Hemoglobin: sufficient oxygen-carrying capacity.
        (new_df['hemoglobin__last__last_12h'].between(10.0, 14.0)) &  # adequate hemoglobin
        # Temperature: normothermia.
        (new_df['temperature__mean__last_12h'].between(36.0, 37.5)) &  # normal body temperature
        # APTT: normal clotting function.
        (new_df['activated_partial_thromboplastin_time__last__last_12h'].between(25.0, 40.0)) &  # normal coagulation range
        # Bicarbonate: normal acid–base status.
        (new_df['bicarbonate_arterial__last__last_12h'].between(22.0, 28.0))    # reflective of balanced acid–base metabolism
    )
    new_df.loc[condition_safe, int_var] = 1

# Risky
def dynamic_risky(new_df, pool, int_var, time_name, t):
    new_df.loc[new_df[time_name] == t, int_var] = 0
    # Risky discharge: patients are borderline stable; these values are acceptable but less optimal.
    condition_risky = (
        # Ventilation: may include patients who are off ventilation or just recently removed.
        (new_df['vent_mode__last__last_12h'].isin(['missing', 'invasive_assisted', 'invasive_controlled'])) &  # includes those with recent ventilation
        # Ventilation time: shorter period since stopping ventilation.
        (new_df['vent_mode__hours_since_last__last_12h'].between(4.0, 8.0)) &  # shorter duration off support may be less stable
        # pCO2: mildly elevated values may be tolerated.
        (new_df['pco2_arterial__mean__last_12h'].between(35.0, 50.0)) &  # slight hypercapnia is borderline acceptable
        # pO2: acceptable oxygenation but with a wider range.
        (new_df['po2_arterial__mean__last_12h'].between(60.0, 350.0)) &  # lower bound decreased, upper bound increased for moderate risk
        # Oxygen flow: moderate oxygen needs.
        (new_df['o2_flow__last__last_12h'].between(0.0, 15.0)) &  # some supplemental oxygen required
        # O2 saturation: slightly lower saturation.
        (new_df['o2_saturation__mean__last_12h'].between(90.0, 95.0)) &  # lower saturation suggests borderline respiratory reserve
        # Respiratory rate: slightly elevated or depressed.
        (new_df['respiratory_rate_measured__mean__last_12h'].between(20.0, 24.0)) &  # elevated rate indicates possible distress
        # Neurological status: moderately impaired.
        (new_df['glasgow_coma_scale_total__last__last_12h'].between(10.0, 13.0)) &  # lower GCS indicates reduced responsiveness
        # Lactate: modestly elevated, hinting at some metabolic stress.
        (new_df['lactate__last__last_12h'].between(2.0, 4.0)) &  # intermediate lactate levels are concerning
        # Urine output: lower output may indicate early kidney stress.
        (new_df['fluid_out_urine__total_value_extrapolated__last_12h'].between(1000, 3000)) &  # reduced urine output is less reassuring
        # Ureum: moderately high values.
        (new_df['ureum__last__last_12h'].between(40.0, 80.0)) &  # higher ureum levels suggest borderline renal function
        # Creatinine: borderline elevated.
        (new_df['creatinine__last__last_12h'].between(1.5, 2.5)) &  # creatinine in a moderate risk range
        # Mean arterial blood pressure: on the lower side.
        (new_df['arterial_blood_pressure_mean__mean__last_12h'].between(60.0, 70.0)) &  # lower blood pressure may be less secure
        # Heart rate: modest tachycardia.
        (new_df['heart_rate__mean__last_12h'].between(100.0, 120.0)) &  # elevated heart rate indicates stress
        # Hemoglobin: borderline low levels.
        (new_df['hemoglobin__last__last_12h'].between(8.0, 10.0)) &  # lower hemoglobin suggests less reserve
        # Temperature: slight hypothermia or fever.
        (new_df['temperature__mean__last_12h'].between(35.0, 36.0)) | (new_df['temperature__mean__last_12h'].between(37.5, 38.0)) &  # mild deviation from normothermia
        # APTT: slightly prolonged clotting time.
        (new_df['activated_partial_thromboplastin_time__last__last_12h'].between(40.0, 60.0)) &  # borderline clotting abnormality
        # Bicarbonate: mild acid–base imbalance.
        (new_df['bicarbonate_arterial__last__last_12h'].between(18.0, 22.0))    # a lower bicarbonate suggests mild acidosis
    )
    new_df.loc[condition_risky, int_var] = 1

# Dangerous
def dynamic_dangerous(new_df, pool, int_var, time_name, t):
    new_df.loc[new_df[time_name] == t, int_var] = 0
    # Dangerous discharge: patients are unstable, and their current parameters predict high 7-day mortality risk if discharged
    condition_dangerous = (
        # Ventilation: patients still requiring mechanical ventilation.
        (new_df['vent_mode__last__last_12h'].isin(['invasive_assisted', 'invasive_controlled'])) &  # active ventilation is a red flag
        # Ventilation time: very short duration since ventilation removal (if any).
        (new_df['vent_mode__hours_since_last__last_12h'].between(0.0, 4.0)) &  # minimal time off ventilatory support is dangerous
        # pCO2: values outside the normal window.
        ((new_df['pco2_arterial__mean__last_12h'] < 30.0) | (new_df['pco2_arterial__mean__last_12h'] > 50.0)) &  # marked derangements in pCO2
        # pO2: critically low oxygenation or excessive levels that may indicate oxygen toxicity.
        ((new_df['po2_arterial__mean__last_12h'] < 60.0) | (new_df['po2_arterial__mean__last_12h'] > 400.0)) &  # dangerous oxygenation levels
        # Oxygen flow: high oxygen supplementation requirement.
        (new_df['o2_flow__last__last_12h'] > 15.0) &  # high flow rates indicate significant respiratory compromise
        # O2 saturation: dangerously low saturations.
        (new_df['o2_saturation__mean__last_12h'] < 90.0) &  # saturation below 90% is worrisome
        # Respiratory rate: extremes of breathing rate.
        ((new_df['respiratory_rate_measured__mean__last_12h'] < 10.0) | (new_df['respiratory_rate_measured__mean__last_12h'] > 24.0)) &  # abnormal respiratory rate indicates high risk
        # Neurological status: significant depression in consciousness.
        (new_df['glasgow_coma_scale_total__last__last_12h'] < 10.0) &  # very low GCS is highly concerning
        # Lactate: high levels reflect tissue hypoperfusion.
        (new_df['lactate__last__last_12h'] > 4.0) &  # markedly elevated lactate suggests shock
        # Urine output: very low output raises concern for renal failure.
        (new_df['fluid_out_urine__total_value_extrapolated__last_12h'] < 1000) &  # very low urine output is dangerous
        # Ureum: high values imply severe renal dysfunction.
        (new_df['ureum__last__last_12h'] > 80.0) &  # high ureum levels are red flags for renal impairment
        # Creatinine: high creatinine indicates poor kidney function.
        (new_df['creatinine__last__last_12h'] > 2.5) &  # creatinine above 2.5 signals significant renal compromise
        # Mean arterial blood pressure: values too low or too high compromise perfusion.
        ((new_df['arterial_blood_pressure_mean__mean__last_12h'] < 60.0) | (new_df['arterial_blood_pressure_mean__mean__last_12h'] > 140.0)) &  # abnormal blood pressure is dangerous
        # Heart rate: excessive tachycardia or bradycardia.
        ((new_df['heart_rate__mean__last_12h'] < 60.0) | (new_df['heart_rate__mean__last_12h'] > 120.0)) &  # heart rate outside normal limits increases risk
        # Hemoglobin: too low levels may result in poor oxygen delivery.
        (new_df['hemoglobin__last__last_12h'] < 8.0) &  # very low hemoglobin is concerning
        # Temperature: significant hypothermia or fever.
        ((new_df['temperature__mean__last_12h'] < 35.0) | (new_df['temperature__mean__last_12h'] > 38.0)) &  # marked temperature derangement is a risk factor
        # APTT: significantly prolonged clotting times.
        (new_df['activated_partial_thromboplastin_time__last__last_12h'] > 60.0) &  # high APTT suggests coagulopathy
        # Bicarbonate: severe acid–base imbalance.
        ((new_df['bicarbonate_arterial__last__last_12h'] < 18.0) | (new_df['bicarbonate_arterial__last__last_12h'] > 30.0))   # extremes in bicarbonate reflect dangerous acid–base status
    )
    new_df.loc[condition_dangerous, int_var] = 1


def super_dynamic_dangerous(new_df, pool, int_var, time_name, t):
    new_df.loc[new_df[time_name] == t, int_var] = 0
    condition_extremely_dangerous = (
        (new_df['glasgow_coma_scale_total__last__last_12h'] <= 6) &  # deep coma or no meaningful response
        (new_df['lactate__last__last_12h'] >= 6.0)  # severe lactic acidosis, reflects profound shock
    )
    new_df.loc[condition_extremely_dangerous, int_var] = 1'''


g = ParametricGformula(obs_data=mimicdata, id=id_, time_name=time_name,
                       covnames=covnames, covtypes=covtypes,
                       covmodels=covmodels, basecovs=basecovs,
                       int_descript=int_descript,
                       time_points=time_points,
                       
                       Intervention1_A=Intervention1_A,
                       #Intervention2_A=Intervention2_A,
                       #Intervention3_A=Intervention3_A,
                       #Intervention4_A=Intervention4_A,
                       #Intervention5_A=Intervention5_A,
                       #Intervention6_A=Intervention6_A,
                       #Intervention7_A=Intervention7_A,
                       
                       #Intervention2_A = [dynamic_NC],
                       #Intervention3_A = [dynamic_safe],
                       #Intervention4_A = [dynamic_risky],
                       #Intervention5_A = [dynamic_dangerous],

                       #Intervention1_A=[super_dynamic_dangerous],
                       
                       outcome_name=outcome_name, ymodel=ymodel,
                       outcome_type=outcome_type,
                       ref_int=0,
                       model_fits=True,
                       n_simul = 500,
                       censor_name = censor_name, censor_model = censor_model
                       #nsamples=20, parallel=True, ncores=8
                       )

%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
{'Discharge 3rd day': [['A', <function static at 0x000002756E862CA0>, array([0, 0, 0, 0, 0, 1]), [0, 1, 2, 3, 4, 5]]], 'Natural course': <function natural at 0x000002756E862C00>}
%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%


In [225]:
g.fit()

start fitting parametric model.
check...
$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$ vent_mode__last__last_12h
Optimization terminated successfully.
         Current function value: 0.442232
         Iterations 14
$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$ vent_mode__hours_since_last__last_12h
$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$ pco2_arterial__mean__last_12h
$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$ po2_arterial__mean__last_12h
$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$ o2_flow__last__last_12h
$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$ o2_saturation__mean__last_12h
$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$ respiratory_rate_measured__mean__last_12h
$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$ glasgow_coma_scale_total__last__last_12h
$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$ lactate__last__last_12h
$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$ fluid_out_urine__mean__last_12h
$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$ ureum__last__last_12h
$$$$$$$$$$$$

# Inspecting Y across LOS

In [226]:
(
    mimicdata.sort_values(['admission_id','t0'])
             .groupby('admission_id')
             .tail(1)['Y']
             .mean()
)

0.014636696288552013

In [227]:
# get last row per admission (with Y and t0)
last_rows = mimicdata.groupby("admission_id").tail(1)[["admission_id", "t0", "Y"]]

# compute LOS (t0 starts at 0, so add 1)
last_rows["LOS"] = last_rows["t0"] + 1

# mean Y per LOS
mean_y_per_los = last_rows.groupby("LOS")["Y"].mean()

print(mean_y_per_los)


LOS
1      0.007642
2      0.006349
3      0.007678
4      0.010601
5      0.029703
         ...   
101         NaN
109    0.000000
111    0.000000
117    0.000000
181    0.000000
Name: Y, Length: 78, dtype: float64


In [228]:
NC = g.summary_dict['sim_data']['Natural course']

# get last row per admission (with Y and t0)
last_rows = NC.groupby("admission_id").tail(1)[["admission_id", "t0", "Py"]]

# compute LOS (t0 starts at 0, so add 1)
last_rows["LOS"] = last_rows["t0"] + 1

# mean Y per LOS
mean_y_per_los = last_rows.groupby("LOS")["Py"].mean()

print(mean_y_per_los)

LOS
1     0.012542
2     0.014475
3     0.011990
4     0.011457
5     0.014993
6     0.013671
7     0.011418
8     0.017295
9     0.021294
10    0.021830
11    0.014760
12    0.023384
13    0.021158
14    0.021067
15    0.020221
Name: Py, dtype: float64


In [229]:
mimicdata.groupby('admission_id').tail(1)['D'].mean()

0.11578460827363068

In [230]:
mimicdata.admission_id.nunique()

4327

In [231]:
mimicdata['D'].unique()

array([0, 1], dtype=int64)

In [232]:
mimicdata[['Y', 'D']].drop_duplicates(ignore_index=True)

,Y,D
0,NaN,0
1,0.0,0
2,NaN,1
3,1.0,0


In [233]:
Amodel = g.summary_dict['all_model_fits']['A']

In [234]:
As = Amodel.model.endog

In [235]:
As

array([1., 0., 0., ..., 0., 1., 1.])

In [236]:
pd.Series(As).value_counts()

0.0    22977
1.0     2910
Name: count, dtype: int64

In [237]:
mimicdata.A.value_counts()

A
0    26388
1     3826
Name: count, dtype: int64

In [238]:
Amodel.params['Intercept']

-20.16521070456725

In [239]:
g.summary_dict['model_fits_summary']['Y']

<class 'statsmodels.iolib.summary.Summary'>
"""
                 Generalized Linear Model Regression Results                  
==============================================================================
Dep. Variable:                      Y   No. Observations:                 3826
Model:                            GLM   Df Residuals:                     3805
Model Family:                Binomial   Df Model:                           20
Link Function:                  Logit   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -264.38
Date:                Fri, 05 Dec 2025   Deviance:                       528.76
Time:                        21:50:55   Pearson chi2:                 4.37e+03
No. Iterations:                    22   Pseudo R-squ. (CS):            0.01441
Covariance Type:            nonrobust                                         
================================================================================================================================
                                                                   coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------------------------------------------------------
Intercept                                                       -4.6327   1838.197     -0.003      0.998   -3607.433    3598.168
vent_mode__last__last_12h[T.invasive_assisted]                   4.7418   3676.394      0.001      0.999   -7200.859    7210.343
vent_mode__last__last_12h[T.invasive_controlled]               -14.1370   9190.986     -0.002      0.999    -1.8e+04     1.8e+04
vent_mode__last__last_12h[T.unknown]                             4.7625   3676.394      0.001      0.999   -7200.838    7210.363
A                                                               -4.6327   1838.197     -0.003      0.998   -3607.433    3598.168
cumavg_vent_mode__hours_since_last__last_12h                    -0.0918      0.159     -0.577      0.564      -0.404       0.220
cumavg_pco2_arterial__mean__last_12h                            -0.0824      0.150     -0.549      0.583      -0.376       0.212
cumavg_po2_arterial__mean__last_12h                             -0.4600      0.163     -2.829      0.005      -0.779      -0.141
cumavg_o2_flow__last__last_12h                                  -0.2361      0.153     -1.542      0.123      -0.536       0.064
cumavg_o2_saturation__mean__last_12h                             0.0569      0.186      0.306      0.759      -0.307       0.421
cumavg_respiratory_rate_measured__mean__last_12h                 0.4697      0.183      2.569      0.010       0.111       0.828
cumavg_glasgow_coma_scale_total__last__last_12h                 -0.1894      0.150     -1.262      0.207      -0.484       0.105
cumavg_lactate__last__last_12h                                   0.0102      0.138      0.074      0.941      -0.259       0.280
cumavg_fluid_out_urine__mean__last_12h                          -0.2304      0.180     -1.278      0.201      -0.584       0.123
cumavg_ureum__last__last_12h                                     0.2856      0.168      1.696      0.090      -0.044       0.616
cumavg_creatinine__last__last_12h                               -0.1471      0.192     -0.766      0.444      -0.524       0.229
cumavg_arterial_blood_pressure_mean__mean__last_12h             -0.1670      0.170     -0.984      0.325      -0.500       0.166
cumavg_heart_rate__mean__last_12h                                0.3981      0.167      2.379      0.017       0.070       0.726
cumavg_hemoglobin__last__last_12h                                0.1359      0.131      1.035      0.301      -0.121       0.393
cumavg_temperature__mean__last_12h                              -0.5561      0.205     -2.712      0.007      -0.958      -0.154
cumavg_activated_partial_thromboplastin_time__last__last_12h    -0.2373      0.165     -1.436      0.151      -0.561

In [240]:
g.summary_dict['model_fits_summary']['A']

<class 'statsmodels.iolib.summary.Summary'>
"""
                 Generalized Linear Model Regression Results                  
==============================================================================
Dep. Variable:                      A   No. Observations:                25887
Model:                            GLM   Df Residuals:                    25866
Model Family:                Binomial   Df Model:                           20
Link Function:                  Logit   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -8252.3
Date:                Fri, 05 Dec 2025   Deviance:                       16505.
Time:                        21:50:55   Pearson chi2:                 3.18e+04
No. Iterations:                    19   Pseudo R-squ. (CS):            0.06339
Covariance Type:            nonrobust                                         
=========================================================================================================================
                                                            coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------------------------------------------------
Intercept                                               -20.1652   1.77e+04     -0.001      0.999   -3.48e+04    3.47e+04
vent_mode__last__last_12h[T.invasive_assisted]           17.5327   1.77e+04      0.001      0.999   -3.47e+04    3.48e+04
vent_mode__last__last_12h[T.invasive_controlled]         15.3687   1.77e+04      0.001      0.999   -3.47e+04    3.48e+04
vent_mode__last__last_12h[T.unknown]                     18.4530   1.77e+04      0.001      0.999   -3.47e+04    3.48e+04
lag1_A                                                -5.493e-09   5.45e-06     -0.001      0.999   -1.07e-05    1.07e-05
vent_mode__hours_since_last__last_12h                     0.0895      0.020      4.545      0.000       0.051       0.128
po2_arterial__mean__last_12h                              0.0945      0.020      4.846      0.000       0.056       0.133
o2_flow__last__last_12h                                   0.0177      0.020      0.882      0.378      -0.022       0.057
o2_saturation__mean__last_12h                            -0.1020      0.021     -4.890      0.000      -0.143      -0.061
respiratory_rate_measured__mean__last_12h                -0.1738      0.024     -7.274      0.000      -0.221      -0.127
glasgow_coma_scale_total__last__last_12h                  0.3021      0.024     12.413      0.000       0.254       0.350
lactate__last__last_12h                                  -0.0078      0.020     -0.387      0.699      -0.047       0.032
fluid_out_urine__mean__last_12h                           0.0625      0.020      3.172      0.002       0.024       0.101
ureum__last__last_12h                                    -0.0646      0.028     -2.281      0.023      -0.120      -0.009
creatinine__last__last_12h                                0.0361      0.028      1.309      0.190      -0.018       0.090
arterial_blood_pressure_mean__mean__last_12h              0.0658      0.020      3.212      0.001       0.026       0.106
heart_rate__mean__last_12h                               -0.1161      0.022     -5.199      0.000      -0.160      -0.072
hemoglobin__last__last_12h                                0.0231      0.020      1.137      0.256      -0.017       0.063
temperature__mean__last_12h                              -0.1648      0.023     -7.151      0.000      -0.210      -0.120
activated_partial_thromboplastin_time__last__last_12h    -0.0013      0.021     -0.060      0.952      -0.042       0.039
bicarbonate_arterial__last__last_12h                     -0.0060      0.021     -0.281      0.778      -0.048       0.036
t0                                                       -0.0340      0.002    -14.336      0.000      -0.039      -0.029
=================================

In [241]:
pred_probs = Amodel.predict()  

In [242]:
pd.Series(pred_probs).describe()

count    2.588700e+04
mean     1.124116e-01
std      7.647305e-02
min      1.170226e-09
25%      4.940734e-02
50%      1.070258e-01
75%      1.673265e-01
max      4.349470e-01
dtype: float64

In [243]:
len(As)

25887

In [244]:
plt.hist(pred_probs, bins=50)
plt.title("Predicted Probabilities of A=1")
plt.xlabel("Probability")
plt.ylabel("Count")
plt.grid(True)
plt.show()

In [245]:
from sklearn.metrics import classification_report, roc_auc_score

# Assume your true values are in `y_true` and predicted probs from `fit.predict()`
y_pred = (pred_probs > 0.2).astype(int)

print(classification_report(As, y_pred))
print("ROC AUC:", roc_auc_score(As, pred_probs))


              precision    recall  f1-score   support

         0.0       0.91      0.88      0.89     22977
         1.0       0.24      0.30      0.26      2910

    accuracy                           0.81     25887
   macro avg       0.57      0.59      0.58     25887
weighted avg       0.83      0.81      0.82     25887

ROC AUC: 0.725779178251911


In [246]:
from sklearn.metrics import average_precision_score
average_precision = average_precision_score(As, pred_probs)
print("Average Precision (PR AUC):", average_precision)


Average Precision (PR AUC): 0.2112607807442839


In [247]:
NC = g.summary_dict['sim_data']['Natural course']

In [248]:
# Compute LOS
los_nc = NC.groupby('admission_id')['t0'].max() + 1
los_mimic = mimicdata.groupby('admission_id')['t0'].max().clip(upper=100) + 1
los_di = DI.groupby('admission_id')['t0'].max() + 1

# Create vertically stacked subplots
fig, axs = plt.subplots(nrows=3, ncols=1, figsize=(10, 10), sharex=True)

# Plot for NC
axs[0].hist(los_nc, bins=range(1, 101), edgecolor='black', color='skyblue')
axs[0].set_title('LOS Distribution: NC')
axs[0].set_ylabel('Number of Admissions')
axs[0].grid(True)

# Plot for mimicdata
axs[1].hist(los_mimic, bins=range(1, 101), edgecolor='black', color='salmon')
axs[1].set_title('LOS Distribution: mimicdata (clipped at 100)')
axs[1].set_xlabel('Length of Stay (number of time points)')
axs[1].set_ylabel('Number of Admissions')
axs[1].grid(True)

# Plot for mimicdata
axs[2].hist(los_di, bins=range(1, 101), edgecolor='black', color='salmon')
axs[2].set_title('LOS Distribution: DI')
axs[2].set_xlabel('Length of Stay (number of time points)')
axs[2].set_ylabel('Number of Admissions')
axs[2].grid(True)

# Final layout
plt.tight_layout()
plt.show()


NameError: name 'DI' is not defined

# Mean Y per LOS

In [ ]:
# Step 1: Get the last row for each admission (i.e., end of follow-up)
last_rows = mimicdata.sort_values("t0").groupby("admission_id").tail(1)

# Step 2: Calculate LOS for each admission
last_rows["los"] = mimicdata.groupby("admission_id")["t0"].nunique().values

# Step 3: Group by LOS and compute mean Y
mean_y_per_los = last_rows.groupby("los")["Y"].mean().reset_index()

# Step 4: Display
print(mean_y_per_los)


In [ ]:
mean_y_per_los.Y.mean()

In [ ]:
IPW = g.summary_dict['IP_weights']

In [ ]:
NC

# Distribution of GCS when A=1

In [ ]:
DI = g.summary_dict['sim_data']['Discharge extremely dangerous']

In [ ]:
DI

In [ ]:
# Filter data where A == 1
subset = DI[DI['A'] == 1]

# Plot distribution of X
plt.figure(figsize=(8, 5))
subset['glasgow_coma_scale_total__last__last_12h'].hist(bins=30, edgecolor='black')
plt.xlabel("L values (when A=1)")
plt.ylabel("Frequency")
plt.title("Distribution of X for A=1")
plt.grid(True)
plt.tight_layout()
plt.show()

In [ ]:
# Filter data where A == 1
subset = NC[NC['A'] == 1]

# Plot distribution of X
plt.figure(figsize=(8, 5))
subset['glasgow_coma_scale_total__last__last_12h'].hist(bins=30, edgecolor='black')
plt.xlabel("L values (when A=1)")
plt.ylabel("Frequency")
plt.title("Distribution of X for A=1")
plt.grid(True)
plt.tight_layout()
plt.show()

In [ ]:
NC

In [ ]:
# Filter rows where A == 1
subset = DI[DI['A'] == 1]

# Loop through each variable in L
for var in picked_for_L:
    print(f"\n--- Variable: {var} ---")
    
    if pd.api.types.is_numeric_dtype(subset[var]):
        # For continuous variable: show min, max, mean
        print("Type: Continuous")
        print("Min:", subset[var].min())
        print("Max:", subset[var].max())
        print("Mean:", round(subset[var].mean(), 2))
    else:
        # For categorical variable: show value counts
        print("Type: Categorical")
        print(subset[var].value_counts(dropna=False))
